In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import bokeh
import hvplot.pandas

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")

In [3]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [4]:
df[df['country'] == 'Indonesia']
# "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
21659,Indonesia,1850,IDN,21979198.0,1.663535e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,96.917,5.171,NaN,NaN
21660,Indonesia,1851,IDN,22178120.0,1.729279e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,3.407,0.000,0.000,0.000,0.000,143.241,5.235,NaN,NaN
21661,Indonesia,1852,IDN,22382251.0,1.685810e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,3.495,0.000,0.000,0.000,0.000,151.931,5.299,NaN,NaN
21662,Indonesia,1853,IDN,22591664.0,1.767151e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,3.638,0.000,0.000,0.000,0.000,168.279,5.364,NaN,NaN
21663,Indonesia,1854,IDN,22806435.0,1.710595e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,3.833,0.000,0.000,0.000,0.000,191.823,5.428,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21828,Indonesia,2019,IDN,272489379.0,3.272568e+12,30.156,0.111,653.792,59.690,10.047,...,NaN,3.401,0.013,0.039,0.053,0.001,2103.968,986.537,10.751,1.644
21829,Indonesia,2020,IDN,274814864.0,3.204973e+12,26.297,0.096,608.223,-45.568,-6.970,...,NaN,3.410,0.014,0.039,0.054,0.001,1682.395,923.432,15.485,2.546
21830,Indonesia,2021,IDN,276758056.0,3.323655e+12,27.262,0.099,619.622,11.399,1.874,...,NaN,3.415,0.014,0.040,0.055,0.001,1648.610,936.604,8.327,1.344
21831,Indonesia,2022,IDN,278830529.0,3.500093e+12,26.843,0.096,737.073,117.450,18.955,...,NaN,3.427,0.014,0.041,0.056,0.001,1817.063,1082.793,-23.360,-3.169


In [5]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)

In [6]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [7]:
df['year'].describe()

count    50191.000000
mean      1919.883067
std         65.627296
min       1750.000000
25%       1875.000000
50%       1924.000000
75%       1974.000000
max       2023.000000
Name: year, dtype: float64

In [8]:
year_slider = pn.widgets.IntSlider(name='Year Slider', start=1750, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year Slider', start=1750, step=5, value=1850)

In [13]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioBoxGroup(
    name = 'Y axis',
    options = ['co2', 'co2_per_capita']
)

In [14]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country','year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [15]:
co2_pipeline

In [16]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='country', y=yaxis_co2,line_width=2, title="CO2 emission by continent")
co2_plot

Traceback (most recent call last):
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\viewable.py", line 480, in _on_msg
 doc.unhold()
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\bokeh\document\document.py", line 776, in unhold
 self.callbacks.unhold()
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\bokeh\document\callbacks.py", line 432, in unhold
 self.trigger_on_change(event)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\bokeh\document\callbacks.py", line 409, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\bokeh\document\callbacks.py", line 444, in invoke_with_curdoc
 return f()
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\bokeh\util\callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\reactive.py", line 478, in _comm_change
 state._handle_exception(e)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\io\state.py", line 458, in _handle_exception
 raise exception
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\reactive.py", line 476, in _comm_change
 self._schedule_change(doc, comm)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\reactive.py", line 458, in _schedule_change
 self._change_event(doc)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\reactive.py", line 454, in _change_event
 self._process_events(events)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\reactive.py", line 391, in _process_events
 self.param.update(**self_params)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 2319, in update
 restore = dict(self_._update(arg, **kwargs))
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 2352, in _update
 self_._batch_call_watchers()
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 2546, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\param.py", line 914, in _replace_pane
 self._update_inner(new_object)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\pane\base.py", line 710, in _update_inner
 new_pane, internal = self._update_from_object(
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\panel\pane\base.py", line 699, in _update_from_object
 old_object.object = object
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 528, in _f
 instance_param.__set__(obj, val)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\parameterized.py", line 1553, in __set__
 obj.param._call_watcher(watcher, event)
 File "c:\users\amfua\documents\python_projects\project_1\project_env\lib\site-packages\param\paramete

In [ ]:
#world.hvplot(geo=True) + world.hvplot(c='continent', geo=True)

#co2_map = co2_pipeline.hvplot(geo=True, title="CO2 emission by continent")
#co2_map


In [17]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table

In [18]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [19]:
co2_vs_gdp_scatterplot_pipeline

In [20]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
co2_vs_gdp_scatterplot

In [21]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [22]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     title='CO2 source by continent')
co2_source_bar_plot

In [32]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.JPG('emissions.jpg', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

In [33]:
template.show()

Launching server at http://localhost:55641
